In [1]:
!pip install scanpy
!pip install leidenalg
!pip install optuna

  Obtaining dependency information for scanpy from https://files.pythonhosted.org/packages/3b/72/436046ca332b933ca7d09cd45b86154232203e068e8307a102d5349e9444/scanpy-1.9.8-py3-none-any.whl.metadata
  Obtaining dependency information for anndata>=0.7.4 from https://files.pythonhosted.org/packages/42/c7/23c2b5d3dd764650d235b574fdcac8d7252e6ae2f90273f65e95b9ca4e34/anndata-0.10.5.post1-py3-none-any.whl.metadata
  Obtaining dependency information for natsort from https://files.pythonhosted.org/packages/ef/82/7a9d0550484a62c6da82858ee9419f3dd1ccc9aa1c26a1e43da3ecd20b0d/natsort-8.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for seaborn>=0.13.0 from https://files.pythonhosted.org/packages/83/11/00d3c3dfc25ad54e731d91449895a79e4bf2384dc3ac01809010ba88f6d5/seaborn-0.13.2-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for array-api-compat from https://files.pythonhosted.org/packages/b8/b7/604e908c9081e901f56186184bbb52ed4ee

In [2]:
import numpy as np
import pandas as pd
import scanpy as sc
import os
import random
import pickle
import sklearn
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist, squareform
from typing import List
## Imports for plotting
import matplotlib.pyplot as plt
import seaborn as sns

## PyTorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from collections import defaultdict
import optuna
import gc

In [ ]:
import warnings
from sklearn.exceptions import DataConversionWarning

# Suppress DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')
SEED = 111
random.seed(SEED)
np.random.seed(SEED)

scanpy==1.9.8 anndata==0.10.5.post1 umap==0.5.5 numpy==1.24.3 scipy==1.11.4 pandas==2.0.3 scikit-learn==1.2.2 statsmodels==0.14.0 igraph==0.11.3 pynndescent==0.5.11


In [ ]:
#dataset='Stephenson'
dataset='COMBAT'

## Data preprocessing

In [4]:
if dataset=='Stephenson':
    # Stephenson Dataset 
    adata = sc.read_h5ad("/kaggle/input/stephenson-et-al-covid-19-single-cell-dataset/Stephenson_processed.h5ad")
    donors = adata.obs['sample_id'].unique()

    # Splitting data into train-test-validation (cant perform cross-validation!)
    test_set= adata[adata.obs['Site']=='Sanger']
    # train_val_set= adata[~(adata.obs['Site']=='Cambridge')]
    train_set= adata[adata.obs['Site']=='Ncl']
    validation_set= adata[adata.obs['Site']=='Cambridge']
    test_patients_ids=test_set.obs['sample_id'].unique()
    layers = ['X_pca', 'X_scVI', 'X_scANVI']
else: 
    # COMBAT dataset 
    
    adata = sc.read_h5ad("/kaggle/input/a-blood-atlas-of-covid-19-combat-preprocessed/combat_processed.h5ad")
    donors = adata.obs['scRNASeq_sample_ID'].unique()

    # Splitting data into test and remaining data for cross-validation
    train_val_set_ids, test_ids = train_test_split(
        adata.obs['Pool_ID'].unique(), 
        test_size=0.2,  # 20% of data for testing
        random_state=42  # For reproducibility
    )

    train_val_set = adata[adata.obs['Pool_ID'].isin(train_val_set_ids)]
    test_set = adata[adata.obs['Pool_ID'].isin(test_ids)]
    test_patients_ids=test_set.obs['scRNASeq_sample_ID'].unique()
    train_val_unique_pool_ids = train_val_set.obs['Pool_ID'].unique()
    layers = ['X_pca', 'X_scVI_Pool_ID', 'X_scANVI_Pool_ID']

AnnData object with n_obs × n_vars = 783704 × 3000
    obs: 'Annotation_cluster_id', 'Annotation_cluster_name', 'Annotation_minor_subset', 'Annotation_major_subset', 'Annotation_cell_type', 'GEX_region', 'QC_ngenes', 'QC_total_UMI', 'QC_pct_mitochondrial', 'QC_scrub_doublet_scores', 'TCR_chain_composition', 'TCR_clone_ID', 'TCR_clone_count', 'TCR_clone_proportion', 'TCR_contains_unproductive', 'TCR_doublet', 'TCR_chain_TRA', 'TCR_v_gene_TRA', 'TCR_d_gene_TRA', 'TCR_j_gene_TRA', 'TCR_c_gene_TRA', 'TCR_productive_TRA', 'TCR_cdr3_TRA', 'TCR_umis_TRA', 'TCR_chain_TRA2', 'TCR_v_gene_TRA2', 'TCR_d_gene_TRA2', 'TCR_j_gene_TRA2', 'TCR_c_gene_TRA2', 'TCR_productive_TRA2', 'TCR_cdr3_TRA2', 'TCR_umis_TRA2', 'TCR_chain_TRB', 'TCR_v_gene_TRB', 'TCR_d_gene_TRB', 'TCR_j_gene_TRB', 'TCR_c_gene_TRB', 'TCR_productive_TRB', 'TCR_chain_TRB2', 'TCR_v_gene_TRB2', 'TCR_d_gene_TRB2', 'TCR_j_gene_TRB2', 'TCR_c_gene_TRB2', 'TCR_productive_TRB2', 'TCR_cdr3_TRB2', 'TCR_umis_TRB2', 'BCR_umis_HC', 'BCR_contig_qc_HC

In [5]:
# nr of patients in the dataset
len(donors)

140

## Contrastive learning

In this part of the notebook, we build a neutron network by adjusting the hyperparameters such as depth (adjustable by **num_layers_values**), number of neurons in a layer (**hidden_size_values**), match size (**batch_sizes**). We also select a different number of cells for each patient to build the pseudo-bulk, which is also an adjustable parameter.

In [6]:
# In this notebook, we use data loaders with heavier computational processing. It is recommended to use as many
# workers as possible in a data loader, which corresponds to the number of CPU cores
NUM_WORKERS = os.cpu_count()
  
# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)
print("Number of workers:", NUM_WORKERS)

Device: cpu
Number of workers: 4


In [11]:
from scipy.sparse import issparse
class SamplesDataset(Dataset):
    def __init__(self, data, unique_categories: List[str], sample_col: str, layer: str):
        """
        A dataset that holds sample cells based on unique categories.

        Args:
            data (YourDataType): The dataset containing the data.
            unique_categories (List[str]): List of unique category values.
            sample_col (str): The column indicating sample information.
            layer (str, optional): The data layer to be used. Defaults to "X_pca".
        """
        self.data = data
        self.unique_categories = unique_categories
        self.sample_cells = []
        self.sample_col = sample_col
        self.layer = layer
        
        for sample_id in self.unique_categories:
            sample = self.data.obsm[layer][self.data.obs[sample_col] == sample_id]
            # Convert sparse matrix to dense, if necessary
            if issparse(sample):
                sample = sample.toarray()  # Convert sparse to dense
            if len(sample) > 0:
                self.sample_cells.append(sample)

    def __len__(self):
        return len(self.sample_cells)

    def __getitem__(self, idx):
        return self.sample_cells[idx]
    

class TransformedPairDataset(Dataset):
    """Create two different subsamples based on one sample from the original dataset."""

    def __init__(self, dataset: Dataset, subset_size: int = 1000, validation = False):
        """
        Initialize a dataset with transformed pairs.
        Args:
            dataset (Dataset): The input dataset.
            subset_size (int, optional): Size of the transformed subset. Defaults to 1000.
        """
        self.dataset = dataset  # The input dataset
        self.subset_size = subset_size  # Size of the transformed subset
        self.validation = validation

    def __len__(self):
        """
        Get the length of the dataset.
        Returns:
            int: The length of the dataset.
        """
        return len(self.dataset)

    def __getitem__(self, idx):
        """
        Get a pair of transformed subsamples based on the original sample.
        Args:
            idx (int): Index of the original sample.
        Returns:
            tuple: A tuple containing two transformed subsamples.
        """
        sample = self.dataset[idx]  # Get the original sample
        
        ##########################
        # Randomly select indices for the transformed subsamples
        if self.validation:
            actual_subset_size = min(sample.shape[0], self.subset_size)
            random_indices_1 = np.random.choice(sample.shape[0], size=actual_subset_size)
            random_indices_2 = np.random.choice(sample.shape[0], size=actual_subset_size)
        else:
            random_indices_1 = np.random.choice(sample.shape[0], size=self.subset_size)
            random_indices_2 = np.random.choice(sample.shape[0], size=self.subset_size)
        ##########################
        
        # Compute the mean of the selected indices to create transformed subsamples
        random_sample_1 = sample[random_indices_1].mean(axis=0)
        random_sample_2 = sample[random_indices_2].mean(axis=0)
        
        return random_sample_1, random_sample_2


class DynamicNetwork(nn.Module):
    def __init__(self, n_input_features: int, n_output_features: int, num_layers: int, hidden_size: int):
        """
        Initialize a dynamic neural network.

        Args:
            n_input_features (int): Number of input features.
            n_output_features (int): Number of output features.
            num_layers (int): Number of hidden layers.
            hidden_size (int): Size of the hidden layers.
        """
        super().__init__()

        layers = []
        for _ in range(num_layers):
            
            # Add a linear layer followed by a ReLU activation
            layers.extend([
                nn.Linear(n_input_features, hidden_size),
                nn.ReLU()
            ])
            n_input_features = hidden_size  # Update the number of input features for the next layer

        # Add the final linear layer
        layers.append(nn.Linear(hidden_size, n_output_features))

        # Create a sequential module with the defined layers
        self.fc = nn.Sequential(*layers) # скелет

    def forward(self, x):
        """
        Forward pass through the neural network.
        Args:
            x (tensor): Input data.
        Returns:
            tensor: Output of the network.
        """
        return self.fc(x)
    
# # network with dropout
# class DynamicNetwork(nn.Module):
#     def __init__(self, n_input_features: int, n_output_features: int, num_layers: int, hidden_size: int):
#         """
#         Initialize a dynamic neural network.

#         Args:
#             n_input_features (int): Number of input features.
#             n_output_features (int): Number of output features.
#             num_layers (int): Number of hidden layers.
#             hidden_size (int): Size of the hidden layers.
#         """
#         super().__init__()

#         layers = []
#         for _ in range(num_layers):
#             # Add a linear layer followed by a ReLU activation and then Dropout
#             layers.extend([
#                 nn.Linear(n_input_features, hidden_size),
#                 nn.BatchNorm1d(hidden_size),
#                 nn.ReLU(),
#                 nn.Dropout(0.1)  # Dropout layer with a dropout probability of 0.2
#             ])
#             n_input_features = hidden_size  # Update the number of input features for the next layer

#         # Add the final linear layer without dropout right before output
#         layers.append(nn.Linear(hidden_size, n_output_features))

#         # Create a sequential module with the defined layers
#         self.fc = nn.Sequential(*layers)

#     def forward(self, x):
#         """
#         Forward pass through the neural network.
#         Args:
#             x (tensor): Input data.
#         Returns:
#             tensor: Output of the network.
#         """
#         return self.fc(x)

class InfoNCECauchy(nn.Module):
    def __init__(self, temperature):
        super().__init__()
        self.temperature = temperature

    def forward(self, features):
        
        # Calculate the batch size by dividing the total feature size by 2
        batch_size = features.size(0) // 2
        
        # Split the input features into two parts: a and b
        a = features[:batch_size]
        b = features[batch_size:]
        
        # Compute the pairwise similarities for same-domain (a-a) and different-domain (a-b, b-b) pairs
        sim_aa = 1 / (torch.cdist(a, a) * self.temperature).square().add(1)
        sim_bb = 1 / (torch.cdist(b, b) * self.temperature).square().add(1)
        sim_ab = 1 / (torch.cdist(a, b) * self.temperature).square().add(1)

        # Calculate the tempered alignment term using the logarithm of the diagonal of sim_ab
        
        tempered_alignment = torch.diag(sim_ab).log_().mean()
        # tempered_alignment = torch.diagonal_copy(sim_ab).log_().mean()

        # Exclude self inner product by masking the diagonal elements
        self_mask = torch.eye(batch_size, dtype=bool, device=sim_aa.device)
        sim_aa.masked_fill_(self_mask, 0.0)
        sim_bb.masked_fill_(self_mask, 0.0)
        
        # Calculate the logsumexp terms for both combinations of similarities
        logsumexp_1 = torch.hstack((sim_ab.T, sim_bb)).sum(1).log_().mean()
        logsumexp_2 = torch.hstack((sim_aa, sim_ab)).sum(1).log_().mean()
        
        # Compute the raw uniformity as the sum of logsumexp terms
        raw_uniformity = logsumexp_1 + logsumexp_2
        
        # Calculate the final loss as the negative difference between tempered alignment and raw uniformity
        loss = -(tempered_alignment - raw_uniformity / 2)
        return loss

In [15]:
!mkdir Models
!mkdir Representations

In [16]:
model_folder = 'Models/'
representation_folder = 'Representations/'

In [18]:
# optuna objective for COMBAT dataset
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
num_epochs = 1000
def objective(trial):
    # Sample hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True)
    weight_decay = trial.suggest_float("weight_decay",1e-7,1e-4 ,log=True)
    temperature=trial.suggest_float("temperature",0.01,1)
    batch_size = trial.suggest_int("batch_size", 4,16)
    num_layers = trial.suggest_int("num_layers", 1,4)
    hidden_size = trial.suggest_int("hidden_size", 30,400)
    cell_subset_size = trial.suggest_int("cell_subset_size", 100,6000)
    
    
    # Early stopping parameters
    patience = 10
    best_loss = np.inf
    epochs_no_improve = 0
    
    fold_validation_losses = []

    # Training loop
    for fold, (train_idx, val_idx) in enumerate(kf.split(train_val_unique_pool_ids)):
        train_pool_ids = train_val_unique_pool_ids[train_idx]
        val_pool_ids = train_val_unique_pool_ids[val_idx]
        train_set = train_val_set[train_val_set.obs['Pool_ID'].isin(train_pool_ids)]
        validation_set = train_val_set[train_val_set.obs['Pool_ID'].isin(val_pool_ids)]
        
        train_dataset = SamplesDataset(train_set, donors, sample_col="scRNASeq_sample_ID",layer=layer)
        validation_dataset = SamplesDataset(validation_set, donors, sample_col="scRNASeq_sample_ID",layer=layer)

        train_pairs_dataset = TransformedPairDataset(train_dataset, subset_size=cell_subset_size)
        train_dataloader = DataLoader(train_pairs_dataset, batch_size=batch_size, shuffle=True)

        validation_pairs_dataset = TransformedPairDataset(validation_dataset, subset_size=cell_subset_size, validation=True)
        validation_dataloader = DataLoader(validation_pairs_dataset, batch_size=batch_size, shuffle=True)

        # Model and optimizer for this fold
        network = DynamicNetwork(n_input_features=adata.obsm[layer].shape[1], n_output_features=30, num_layers=num_layers, hidden_size=hidden_size)
        network.to(device)
        optimizer = optim.Adam(network.parameters(), lr=learning_rate, weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, threshold=0.01)
        criterion = InfoNCECauchy(temperature)
        epoch_train_losses = [] # for each epoch
        epoch_validation_losses = [] # for each epoch       

        for epoch in range(num_epochs):
            training_loss = 0  # for each batch
            network.train()
            for batch_idx, (batch) in enumerate(train_dataloader):
                optimizer.zero_grad()
                data1, data2 = batch
                
                samples = torch.vstack((data1, data2)).to(device)
                features = network(samples)
                loss = criterion(features)
                loss.backward()
                optimizer.step()
                
                training_loss += loss.item()
            
            validation_loss = 0
            network.eval()
            with torch.no_grad():  # No gradient computation for validation pass
                for batch_idx, (batch) in enumerate(validation_dataloader):
                    data1, data2 = batch
                    samples = torch.vstack((data1, data2)).to(device)
                    features = network(samples)
                    loss = criterion(features)
                    validation_loss += loss.item()

            avg_training_loss = training_loss / len(train_dataloader)
            avg_validation_loss = validation_loss / len(validation_dataloader)
            epoch_train_losses.append(avg_training_loss)
            epoch_validation_losses.append(avg_validation_loss)
            
            # Early stopping check
            if avg_validation_loss < best_loss:
                best_loss = avg_validation_loss
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print(f"Early stopping triggered at epoch {epoch + 1}")
                    break
            scheduler.step(avg_validation_loss)
        
        fold_validation_losses.append(np.mean(epoch_validation_losses))
        del train_dataset, validation_dataset, train_pairs_dataset, train_dataloader, validation_pairs_dataset, validation_dataloader, network, optimizer, scheduler
        torch.cuda.empty_cache()  # Clear CUDA cache
        gc.collect()
        
    average_validation_loss = np.mean(fold_validation_losses)
    return average_validation_loss

# Create a study object and specify the optimization direction
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)  # Run 100 trials

# Print the best hyperparameters
print('Best hyperparameters:', study.best_params)

# Convert all trials into a DataFrame
df_trials = study.trials_dataframe()

# Assuming 'minimize' direction; for 'maximize', use ascending=False
df_sorted = df_trials.sort_values(by="value", ascending=True)

# Select top N trials - Change N to your desired number of top trials
N = 5
top_n_trials = df_sorted.head(N)
# After reviewing the top N trials, you can proceed with reinitializing and retraining your model
# with the best hyperparameters if you decide that's the best course of action
best_trial = study.best_trial
best_params = best_trial.params
# After the study, identify and save the best trial
best_trial = study.best_trial
# Reinitialize and retrain the model with best hyperparameters
best_params = best_trial.params
# #########################################

[I 2024-03-04 22:41:14,651] A new study created in memory with name: no-name-5d7a5779-1932-45c7-b720-6349f8113202


Early stopping triggered at epoch 55
Early stopping triggered at epoch 1
Early stopping triggered at epoch 1
Early stopping triggered at epoch 1
Early stopping triggered at epoch 1


[I 2024-03-04 22:42:00,504] Trial 0 finished with value: 1.1281214681132272 and parameters: {'learning_rate': 0.0036471112054958735, 'weight_decay': 1.4361184324857036e-06, 'temperature': 0.7322186984904779, 'batch_size': 9, 'num_layers': 2, 'hidden_size': 63, 'cell_subset_size': 2145}. Best is trial 0 with value: 1.1281214681132272.


Early stopping triggered at epoch 48
Early stopping triggered at epoch 1
Early stopping triggered at epoch 1
Early stopping triggered at epoch 1
Early stopping triggered at epoch 1


[I 2024-03-04 22:42:37,972] Trial 1 finished with value: 0.26811918130765355 and parameters: {'learning_rate': 0.00918464301599831, 'weight_decay': 4.005625708860381e-07, 'temperature': 0.6926466819344572, 'batch_size': 6, 'num_layers': 1, 'hidden_size': 110, 'cell_subset_size': 1652}. Best is trial 1 with value: 0.26811918130765355.


...

Early stopping triggered at epoch 17
Early stopping triggered at epoch 1
Early stopping triggered at epoch 1
Early stopping triggered at epoch 1
Early stopping triggered at epoch 1


[I 2024-03-04 23:43:57,008] Trial 98 finished with value: 0.024564752558700175 and parameters: {'learning_rate': 0.08720309705852915, 'weight_decay': 1.628781865561087e-05, 'temperature': 0.7619204481768905, 'batch_size': 4, 'num_layers': 1, 'hidden_size': 206, 'cell_subset_size': 5232}. Best is trial 53 with value: 0.016907496941395296.


Early stopping triggered at epoch 25
Early stopping triggered at epoch 1
Early stopping triggered at epoch 1
Early stopping triggered at epoch 1
Early stopping triggered at epoch 1


[I 2024-03-04 23:44:34,457] Trial 99 finished with value: 0.025827082089015418 and parameters: {'learning_rate': 0.08608301172151576, 'weight_decay': 1.8151073864430793e-05, 'temperature': 0.7956591522884167, 'batch_size': 4, 'num_layers': 1, 'hidden_size': 122, 'cell_subset_size': 5307}. Best is trial 53 with value: 0.016907496941395296.


Best hyperparameters: {'learning_rate': 0.09881731201158052, 'weight_decay': 5.308831706373277e-06, 'temperature': 0.8328467821557369, 'batch_size': 5, 'num_layers': 1, 'hidden_size': 137, 'cell_subset_size': 5987}


In [ ]:
with open('optuna_top_n_trial.pkl', 'wb') as file:
    pickle.dump(top_n_trials, file)

In [ ]:
best_hyperparameters=best_params
best_hyperparameters

In [ ]:
# training and creating representations 
layer_losses_dict={}
for layer in layers:
    if layer=='X_pca':
        best_hyp_list=X_pca_trials
    if layer=='X_scANVI_Pool_ID':
        best_hyp_list=X_scANVI_Pool_ID_trials
    if layer=='X_scVI_Pool_ID':
        best_hyp_list=X_scVI_Pool_ID_trials
    
    if dataset=='COMBAT':
        # whole data
        data_mean_for_all_cells= SamplesDataset(adata, donors, sample_col="scRNASeq_sample_ID",layer=layer)
        #test data only!
        #data_mean_for_all_cells= SamplesDataset(test_set, donors, sample_col="scRNASeq_sample_ID",layer=layer)
    else:
        # whole data
        data_mean_for_all_cells= SamplesDataset(adata, donors, sample_col="sample_id",layer=layer)
        #test data only!
        #data_mean_for_all_cells= SamplesDataset(test_set, donors, sample_col="sample_id",layer=layer)


    means_for_all_cells = []
    # Iterate through the dataset to calculate the mean of each data sample along the rows (axis=0)
    for data in data_mean_for_all_cells:
        means_for_all_cells.append(data.mean(axis=0))
    # Convert the list of means into a NumPy array and then into a PyTorch tensor
    means_for_all_cells = torch.tensor(np.array(means_for_all_cells)).to(device)


    # iterate over best hypterparameter sets from optuna
    #for optuna_trial, row in top_n_trials.iterrows():
    # grid search but with cell subset size and kfold cross val
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    learning_rate = best_hyp_list.get('learning_rate')
    batch_size = best_hyp_list.get('batch_size')
    num_epochs = 200
    num_layers=best_hyp_list.get('num_layers')
    hidden_size = best_hyp_list.get('hidden_size')
    cell_subset_size =best_hyp_list.get('cell_subset_size')
    weight_decay =best_hyp_list.get('weight_decay')
    temperature =best_hyp_list.get('temperature')
    losses_dict = {}
    fold_train_loss=[]
    fold_validation_loss=[]
    
    if dataset=='COMBAT':
        # Iterate over folds
        for fold, (train_idx, val_idx) in enumerate(kf.split(train_val_unique_pool_ids)):
            # Initialize the network and optimizer once for each hyperparameter combination
            network = DynamicNetwork(n_input_features=adata.obsm[layer].shape[1], 
                                     n_output_features=30, num_layers=num_layers, hidden_size=hidden_size)
            network.to(device)
            optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate, weight_decay=weight_decay)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
            criterion = InfoNCECauchy(temperature)
            # Splitting the dataset and creating dataloaders for this fold
            train_pool_ids = train_val_unique_pool_ids[train_idx]
            val_pool_ids = train_val_unique_pool_ids[val_idx]
            train_set = train_val_set[train_val_set.obs['Pool_ID'].isin(train_pool_ids)]
            validation_set = train_val_set[train_val_set.obs['Pool_ID'].isin(val_pool_ids)]

            train_dataset = SamplesDataset(train_set, donors, sample_col="scRNASeq_sample_ID",layer=layer)
            validation_dataset = SamplesDataset(validation_set, donors, sample_col="scRNASeq_sample_ID",layer=layer)

            train_pairs_dataset = TransformedPairDataset(train_dataset, subset_size=cell_subset_size)
            validation_pairs_dataset = TransformedPairDataset(validation_dataset, subset_size=cell_subset_size, validation=True)

            train_dataloader = DataLoader(train_pairs_dataset, batch_size=batch_size, shuffle=True)
            validation_dataloader = DataLoader(validation_pairs_dataset, batch_size=batch_size, shuffle=True)
    else: 
        pair_dataset=TransformedPairDataset(dataset, subset_size=cell_subset_size)
        fold = 99
        network = DynamicNetwork(n_input_features=adata.obsm[layer].shape[1], 
                             n_output_features=30, num_layers=num_layers, hidden_size=hidden_size)
        network.to(device)
        optimizer = torch.optim.Adam(network.parameters(), lr=learning_rate, weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
        criterion = InfoNCECauchy(temperature)
        dataloader=DataLoader(pair_dataset, batch_size=batch_size, shuffle=True)


    # Initialize lists to store losses for this fold
    epoch_train_losses = []
    epoch_validation_losses = []

    # Training loop
    for epoch in range(num_epochs):
        training_loss = 0
        network.train()
        for batch_idx, (batch) in enumerate(train_dataloader):
            optimizer.zero_grad()
            data1, data2 = batch
            samples = torch.vstack((data1, data2)).to(device)
            features = network(samples)
            loss = criterion(features)
            loss.backward()
            optimizer.step()
            training_loss += loss.item()
        validation_loss = 0
        network.eval()
        with torch.no_grad():
            for batch_idx, (batch) in enumerate(validation_dataloader):
                data1, data2 = batch
                samples = torch.vstack((data1, data2)).to(device)
                features = network(samples)
                loss = criterion(features)
                validation_loss += loss.item()

        avg_training_loss = training_loss / len(train_dataloader)
        avg_validation_loss = validation_loss / len(validation_dataloader)
        epoch_train_losses.append(avg_training_loss)
        epoch_validation_losses.append(avg_validation_loss)
        # Update scheduler
        scheduler.step()

    # Append this fold loss to the fold lists
    fold_train_loss.append(epoch_train_losses)
    fold_validation_loss.append(epoch_validation_losses)

    # Compute average losses across folds
    avg_train_losses = [sum(x)/len(x) for x in zip(*fold_train_loss)]
    avg_validation_losses = [sum(x)/len(x) for x in zip(*fold_validation_loss)]

    # Store the averaged losses in the losses_dict
    config_key = (cell_subset_size,fold, batch_size, num_layers, hidden_size, learning_rate, weight_decay)
    losses_dict[config_key] = (avg_train_losses, avg_validation_losses)
    # Save the model and representation for this hyperparameter combination
    model_name = f"{layer}_model_{cell_subset_size}_{fold}_{batch_size}_{num_layers}_{hidden_size}_{num_epochs}_{learning_rate}_{weight_decay}.pt"

    save_path = os.path.join(model_folder, model_name)
    torch.save(network.state_dict(), save_path)

    representation = []
    with torch.no_grad():
        representation = network(means_for_all_cells).cpu().detach().numpy()

    # Save the representation as a CSV file
    representation_df = pd.DataFrame(representation)
    representation_name = f"{layer}_representation_{cell_subset_size}_{fold}_{batch_size}_{num_layers}_{hidden_size}_{num_epochs}_{learning_rate}_{weight_decay}.csv"
    representation_path = os.path.join(representation_folder, representation_name)
    representation_df.to_csv(representation_path, index=False)
    layer_losses_dict[layer]=losses_dict
    with open(f"losses_dict.pkl", 'wb') as file:
        pickle.dump(layer_losses_dict, file)
    output = {}
    for key, subdict in layer_losses_dict.items():
        output[key] = {}
        for subkey, (array1, array2) in subdict.items():
            avg1 =  sum(array1) / len(array1)
            avg2 =  sum(array2) / len(array2)
            output[key][subkey] = (avg1, avg2)
            print(output[key][subkey])
    with open(f"avg_losses_dict.pkl", 'wb') as file:
        pickle.dump(output, file)
